<a href="https://colab.research.google.com/github/ZKTKZ/blender/blob/master/BLENDER_2_7GB_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/ParlAI.git
%cd ParlAI
!python setup.py develop
!pip install 'git+https://github.com/rsennrich/subword-nmt.git#egg=subword-nmt'
!pip install fvcore==0.1.1.post20200716
!pip install Sphinx==3.0 --force-reinstall

In [ ]:
#Interactive
!python parlai/scripts/safe_interactive.py -t blended_skill_talk -mf zoo:blender/blender_3B/model --include_personas=False 
!#python parlai/scripts/interactive.py -mf zoo:blender/blender_3B/model 

In [ ]:
'''
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.10 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: botocore 1.17.48 has requirement docutils<0.16,>=0.10, but you'll have docutils 0.16 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: parlai 0.9.0 has requirement attrs~=19.3, but you'll have attrs 20.1.0 which is incompatible.
ERROR: parlai 0.9.0 has requirement docutils<0.16, but you'll have docutils 0.16 which is incompatible.
'''

!pip install urllib3==1.24 requests==2.23 folium==0.2.1 docutils==0.15 imgaug==0.2.6 attrs==19.3 markdown-it-py~=0.5.2 gitdb==4.0.1 pyflakes==2.2.0

In [ ]:
!parlai display_data -t empathetic_dialogues 

00:01:26 INFO | Opt:
00:01:26 INFO |     batchsize: 1
00:01:26 INFO |     datapath: /content/ParlAI/data
00:01:26 INFO |     datatype: train:ordered
00:01:26 INFO |     dict_class: None
00:01:26 INFO |     display_ignore_fields: agent_reply
00:01:26 INFO |     display_verbose: False
00:01:26 INFO |     download_path: None
00:01:26 INFO |     dynamic_batching: None
00:01:26 INFO |     hide_labels: False
00:01:26 INFO |     image_cropsize: 224
00:01:26 INFO |     image_mode: raw
00:01:26 INFO |     image_size: 256
00:01:26 INFO |     init_model: None
00:01:26 INFO |     init_opt: None
00:01:26 INFO |     loglevel: info
00:01:26 INFO |     max_display_len: 1000
00:01:26 INFO |     model: None
00:01:26 INFO |     model_file: None
00:01:26 INFO |     multitask_weights: [1]
00:01:26 INFO |     num_examples: 10
00:01:26 INFO |     override: "{'task': 'empathetic_dialogues'}"
00:01:26 INFO |     parlai_home: /content/ParlAI
00:01:26 INFO |     remove_political_convos: False
00:01:26 INFO |    

In [ ]:
#Finetune
!parlai train_model -t  blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues --multitask-weights 1,3,3,9 -bs 1 --dropout 0.1 --model-parallel False --skip-generation True --fp16 True --fp16-impl mem_efficient --optimizer adam --learningrate 7e-08 --warmup-updates -100 --update-freq 2 -mf zoo:blender/reddit_3B/model -veps 0.25 --attention-dropout 0.0 --model transformer/generator
#!parlai train_model -t blended_skill_talk,wizard_of_wikipedia,convai2:normalized,empathetic_dialogues --multitask-weights 1,3,3,9 -veps 0.25 --attention-dropout 0.0 -bs 1 --model transformer/generator --embedding-size 2560 --ffn-size 10240 --variant prelayernorm --n-heads 32 --n-positions 128 --n-encoder-layers 2 --n-decoder-layers 24 --history-add-global-end-token end --delimiter '  ' --dict-tokenizer bytelevelbpe  --dropout 0.1 --fp16 True --init-model zoo:blender/reddit_3B/model --dict-file zoo:blender/reddit_3B/model.dict --label-truncate 128 --log_every_n_secs 10 -lr 7e-07 --lr-scheduler reduceonplateau --lr-scheduler-patience 3 --optimizer adam --relu-dropout 0.0 --activation gelu --model-parallel true --save-after-valid True --text-truncate 128 --truncate 128 --warmup_updates 100 --fp16-impl mem_efficient --update-freq 2 --gradient-clip 0.1 --skip-generation True -vp 10 -vmt ppl -vmm min --model-file /tmp/test_train_27B

In [ ]:
# colab free can't finetune 2.7GB :(
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Copied from Parl's docs on /recipes. Possibly outdated, hence left out.

-veps 0.25 --attention-dropout 0.0 --model transformer/generator --embedding-size 2560 --ffn-size 10240 --variant prelayernorm --n-heads 32 --n-positions 128 --n-encoder-layers 2 --n-decoder-layers 24 --history-add-global-end-token end --delimiter '  ' --dict-tokenizer bytelevelbpe  --dropout 0.1 --init-model zoo:blender/reddit_3B/model --dict-file zoo:blender/reddit_3B/model.dict --label-truncate 128 --log_every_n_secs 10 --lr-scheduler reduceonplateau --lr-scheduler-patience 3 --relu-dropout 0.0 --activation gelu --save-after-valid True --text-truncate 128 --truncate 128 --gradient-clip 0.1 --skip-generation True -vp 10 -vmt ppl -vmm min --model-file /tmp/test_train_27B